In [99]:
!pip install nltk sympy
!pip install emoji
!pip install pyspellchecker

In [100]:
import nltk
import os
import re
import string
import emoji
from spellchecker import SpellChecker
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import spacy

In [101]:
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_r

True

In [104]:
from google.colab import files
uploaded = files.upload()  # ستظهر نافذة لاختيار الملف من جهازك

# ثم اقرأ الملف
import pandas as pd
import io

# Get the actual filename from the uploaded dictionary
# Assuming there's only one file uploaded
filename = list(uploaded.keys())[0]

# Read the file using the correct filename
df = pd.read_csv(io.BytesIO(uploaded[filename]))
print(df.head())

Saving BA_AirlineReviews.csv to BA_AirlineReviews (3).csv
   Unnamed: 0  OverallRating                              ReviewHeader  \
0           0            1.0    "Service level far worse then Ryanair"   
1           1            3.0  "do not upgrade members based on status"   
2           2            8.0             "Flight was smooth and quick"   
3           3            1.0             "Absolutely hopeless airline"   
4           4            1.0        "Customer Service is non existent"   

               Name            Datetime  VerifiedReview  \
0           L Keele  19th November 2023            True   
1      Austin Jones  19th November 2023            True   
2        M A Collie  16th November 2023           False   
3        Nigel Dean  16th November 2023            True   
4  Gaylynne Simpson  14th November 2023           False   

                                          ReviewBody TypeOfTraveller  \
0  4 Hours before takeoff we received a Mail stat...  Couple Leisure  

In [105]:
df.columns

Index(['Unnamed: 0', 'OverallRating', 'ReviewHeader', 'Name', 'Datetime',
       'VerifiedReview', 'ReviewBody', 'TypeOfTraveller', 'SeatType', 'Route',
       'DateFlown', 'SeatComfort', 'CabinStaffService', 'GroundService',
       'ValueForMoney', 'Recommended', 'Aircraft', 'Food&Beverages',
       'InflightEntertainment', 'Wifi&Connectivity'],
      dtype='object')

In [106]:
df. info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3701 entries, 0 to 3700
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             3701 non-null   int64  
 1   OverallRating          3696 non-null   float64
 2   ReviewHeader           3701 non-null   object 
 3   Name                   3701 non-null   object 
 4   Datetime               3701 non-null   object 
 5   VerifiedReview         3701 non-null   bool   
 6   ReviewBody             3701 non-null   object 
 7   TypeOfTraveller        2930 non-null   object 
 8   SeatType               3699 non-null   object 
 9   Route                  2926 non-null   object 
 10  DateFlown              2923 non-null   object 
 11  SeatComfort            3585 non-null   float64
 12  CabinStaffService      3574 non-null   float64
 13  GroundService          2855 non-null   float64
 14  ValueForMoney          3700 non-null   float64
 15  Reco

In [107]:
# قائمة الأعمدة المطلوب حذفها (جميع الأعمدة ما عدا VerifiedReview و ReviewBody)
columns_to_drop = [col for col in df.columns if col not in ['VerifiedReview', 'ReviewBody']]

# حذف الأعمدة
df_cleaned = df.drop(columns=columns_to_drop)

# عرض النتيجة
print(df_cleaned.head())

   VerifiedReview                                         ReviewBody
0            True  4 Hours before takeoff we received a Mail stat...
1            True  I recently had a delay on British Airways from...
2           False  Boarded on time, but it took ages to get to th...
3            True  5 days before the flight, we were advised by B...
4           False  We traveled to Lisbon for our dream vacation, ...


In [108]:
VerifiedReview = df_cleaned['VerifiedReview']
df_cleaned.drop('VerifiedReview', axis=1, inplace=True)
df_cleaned['VerifiedReview']=VerifiedReview
df_cleaned.head()

,ReviewBody,VerifiedReview
0,4 Hours before takeoff we received a Mail stat...,True
1,I recently had a delay on British Airways from...,True
2,"Boarded on time, but it took ages to get to th...",False
3,"5 days before the flight, we were advised by B...",True
4,"We traveled to Lisbon for our dream vacation, ...",False


In [109]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df_cleaned['VerifiedReview'] = le.fit_transform(df_cleaned['VerifiedReview'])
df_cleaned.head()

,ReviewBody,VerifiedReview
0,4 Hours before takeoff we received a Mail stat...,1
1,I recently had a delay on British Airways from...,1
2,"Boarded on time, but it took ages to get to th...",0
3,"5 days before the flight, we were advised by B...",1
4,"We traveled to Lisbon for our dream vacation, ...",0


In [110]:
# تحميل نموذج spaCy
nlp = spacy.load("en_core_web_sm")

In [111]:
# Custom stopwords
custom_stopwords = set(stopwords.words('english'))

# Initialize spell checker
spell = SpellChecker()

In [112]:

def remove_punctuation(text):
    punctuationfree = "".join([i if i not in string.punctuation else ' ' for i in text])
    return punctuationfree

In [113]:

def to_lowercase(text):
    return text.lower()

In [114]:

def tokenize(text):
    doc = nlp(text)
    return [token.text for token in doc]

In [115]:

def remove_stopwords(tokens):
    return [token for token in tokens if token not in custom_stopwords]

In [116]:

def lemmatize(tokens):
    doc = nlp(" ".join(tokens))
    return [token.lemma_ for token in doc]

In [117]:
def correct_spelling(tokens):
    corrected_tokens = []
    for token in tokens:
        corrected_token = spell.correction(token)
        if corrected_token is not None:
            corrected_tokens.append(corrected_token)
    return corrected_tokens

In [118]:
# 7. Remove Special Characters and Numbers
def remove_special_chars_and_numbers(tokens):
    # هذا سيقوم بإزالة أي شيء ليس حرفًا أبجديًا أو أرقامًا أو رموزًا رياضية ضرورية
    # Escaping the hyphen or repositioning it within allowed_chars will fix the error.
    allowed_chars = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789+\\-*\/=()^"
    cleaned_tokens = [re.sub(rf'[^{allowed_chars}]', '', token) for token in tokens]
    return [token for token in cleaned_tokens if token]

In [119]:

# 8. Handling Abbreviations (New Preprocessing Step)
abbreviations_dict = {
    "u": "you",
    "r": "are",
    "2": "to",
    "4": "for",
    "ur": "your",
    "txt": "text",
    "msg": "message"
}

def expand_abbreviations(tokens):
    return [abbreviations_dict.get(token, token) for token in tokens]

In [120]:
# 9. Handle Emojis and URLs
def handle_emojis_and_urls(text):
    text = emoji.demojize(text)
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    return text

In [121]:
# 10. Parallel Processing for Preprocessing
def preprocess_text(text):
    text = handle_emojis_and_urls(text)
    text = remove_punctuation(text)
    text = to_lowercase(text)
    tokens = tokenize(text)
    tokens = remove_stopwords(tokens)
    tokens = lemmatize(tokens)
    tokens = correct_spelling(tokens)
    tokens = remove_special_chars_and_numbers(tokens)
    tokens = expand_abbreviations(tokens)
    return tokens

In [ ]:

df_cleaned['ReviewBody'] = df['ReviewBody'].apply(preprocess_text)
df_cleaned.head()

In [ ]:
# initialize NLTK sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

def get_sentiment(text):
    scores = analyzer.polarity_scores(text)
    sentiment = 1 if scores['compound'] >= 0.05 else (0 if scores['compound'] <= -0.05 else 2)
    return sentiment

In [ ]:
# apply get_sentiment function
df_cleaned['sentiment'] = df['ReviewBody'].apply(get_sentiment)
df_cleaned.head()

In [ ]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(df_cleaned['VerifiedReview'], df_cleaned['sentiment']))

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(df_cleaned['VerifiedReview'], df_cleaned['sentiment']))